# German Trafic Sign Detection using Tensorflow

In [2]:
import json
import zipfile
import os

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

os.environ['KAGGLE_USERNAME'] = ''
os.environ['KAGGLE_KEY'] = ''
api_token = {"username":"","key":""}

with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)
!chmod 600 /content/.kaggle/kaggle.json
!kaggle config set -n path -v /content

mkdir: cannot create directory ‘/root/.kaggle’: File exists
Expecting value: line 1 column 1 (char 0)


In [3]:
!kaggle datasets list -s "German Traffic Sign Recognition"

ref                                                                         title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign                     GTSRB - German Traffic Sign Recognition Benchmark  612MB  2018-11-25 18:12:34          51865        789  0.8235294        
datasets/valentynsichkar/traffic-signs-preprocessed                         Traffic Signs Preprocessed                           4GB  2019-08-31 18:22:11           7618        220  1.0              
datasets/eunjurho/german-traffic-sign-recognition-benchmark-cropped         german_traffic_sign_recognition_benchmark_cropped  206MB  2019-11-10 15:22:13            258          6  0.75             
datas

In [4]:
!kaggle datasets download -d "meowmeowmeowmeowmeow/gtsrb-german-traffic-sign"

100% 610M/612M [00:04<00:00, 146MB/s]
100% 612M/612M [00:04<00:00, 137MB/s]


In [ ]:
!unzip '/content/gtsrb-german-traffic-sign.zip' -d '/content/dataset/'

In [6]:
# This command won't be needed all the time. 
# There were some extra copied folder after the unzip operation. That is why this command is used 
# to delete those "same" folder

#!rm -rf '/content/dataset/meta'
#!rm -rf '/content/dataset/train'
#!rm -rf '/content/dataset/test'

### Dataset provides data for train and test
### But the training folder needs to be converted into training and validation.

In [7]:
import os
import glob
from sklearn.model_selection import train_test_split
import shutil

def split_data(path_to_data, path_to_train, path_to_val, split_size=0.1):

  # Provides all folder names from a path
  folders = os.listdir(path_to_data)

  for folder in folders:
    full_path = os.path.join(path_to_data, folder)
    image_path = glob.glob(os.path.join(full_path, '*.png'))

    x_train, x_val = train_test_split(image_path, test_size=split_size)

    for x in x_train:
      basename = os.path.basename(x)
      path_to_folder = os.path.join(path_to_train, folder)

      if not os.path.isdir(path_to_folder):
        os.makedirs(path_to_folder)
      
      shutil.copy(x, path_to_folder)

    for x in x_val:
      basename = os.path.basename(x)
      path_to_folder = os.path.join(path_to_val, folder)

      if not os.path.isdir(path_to_folder):
        os.makedirs(path_to_folder)
      
      shutil.copy(x, path_to_folder)



    

In [8]:
path_to_data = '/content/dataset/Train'
path_to_train = '/content/dataset/training_data_split/train'
path_to_val = '/content/dataset/training_data_split/val'

In [9]:
split_data(path_to_data=path_to_data, path_to_train=path_to_train, path_to_val=path_to_val)

### Time to rearrange Test set

In [10]:
import csv

def order_test_set(path_to_images, path_to_csv):

    try:
        with open(path_to_csv, 'r') as csvfile:

            reader = csv.reader(csvfile, delimiter=',')

            for i, row in enumerate(reader):

                if i==0:
                    continue

                img_name = row[-1].replace('Test/', '')
                label = row[-2]

                path_to_folder = os.path.join(path_to_images, label)

                if not os.path.isdir(path_to_folder):
                    os.makedirs(path_to_folder)

                img_full_path = os.path.join(path_to_images, img_name)
                shutil.move(img_full_path, path_to_folder)

    except:
        print('[INFO] : Error reading csv file')


In [11]:
path_to_images = '/content/dataset/Test'
path_to_csv = '/content/dataset/Test.csv'

order_test_set(path_to_images=path_to_images, path_to_csv=path_to_csv)

## Functional Model

In [20]:
from tensorflow.keras.layers import Conv2D, Input, Dense, MaxPool2D, BatchNormalization, GlobalAvgPool2D, Flatten
from tensorflow.keras import Model

def streesigns_model(nbr_classes):

    my_input = Input(shape=(60,60,3))

    x = Conv2D(32, (3,3), activation='relu')(my_input)
    x = MaxPool2D()(x)
    x = BatchNormalization()(x)

    x = Conv2D(64, (3,3), activation='relu')(x)
    x = MaxPool2D()(x)
    x = BatchNormalization()(x)

    x = Conv2D(128, (3,3), activation='relu')(x)
    x = MaxPool2D()(x)
    x = BatchNormalization()(x)

    # x = Flatten()(x)
    x = GlobalAvgPool2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(nbr_classes, activation='softmax')(x)

    return Model(inputs=my_input, outputs=x)

# Data Generator
**Generators are responsible to set structure for labels. This dataset is now categorized into folders. But there are no labels. Data generator takes care of this. It can take the folder structure and prepare for model training without generating label array.**

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def create_generators(batch_size, train_data_path, val_data_path, test_data_path):

    train_preprocessor = ImageDataGenerator(
        rescale = 1 / 255.,
        rotation_range=10,
        width_shift_range=0.1
    )

    test_preprocessor = ImageDataGenerator(
        rescale = 1 / 255.,
    )

    train_generator = train_preprocessor.flow_from_directory(
        train_data_path,
        class_mode="categorical",
        target_size=(60,60),
        color_mode='rgb',
        shuffle=True,
        batch_size=batch_size
    )

    val_generator = test_preprocessor.flow_from_directory(
        val_data_path,
        class_mode="categorical",
        target_size=(60,60),
        color_mode="rgb",
        shuffle=False,
        batch_size=batch_size,
    )

    test_generator = test_preprocessor.flow_from_directory(
        test_data_path,
        class_mode="categorical",
        target_size=(60,60),
        color_mode="rgb",
        shuffle=False,
        batch_size=batch_size,
    )

    return train_generator, val_generator, test_generator

In [15]:
path_to_train = '/content/dataset/training_data_split/train'
path_to_val = '/content/dataset/training_data_split/val'
path_to_test = '/content/dataset/Test'
batch_size = 64

train_generator, val_generator, test_generator = create_generators(batch_size=batch_size, train_data_path=path_to_train, val_data_path=path_to_val,
                  test_data_path=path_to_test)


Found 35288 images belonging to 43 classes.
Found 3921 images belonging to 43 classes.
Found 12630 images belonging to 43 classes.


In [22]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

path_to_save_model = '/content/Models'
ckpt_saver = ModelCheckpoint(
    path_to_save_model,
    monitor="val_accuracy",
    mode='max',
    save_best_only=True,
    save_freq='epoch',
    verbose=1
)

early_stop = EarlyStopping(monitor="val_accuracy", patience=10)

nbr_classes = train_generator.num_classes
epochs = 1

model = streesigns_model(nbr_classes)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_generator,
          epochs=epochs,
          batch_size=batch_size,
          validation_data=val_generator,
          callbacks=[ckpt_saver, early_stop]
          )



552/552 [==============================] - ETA: 0s - loss: 1.6038 - accuracy: 0.5409
Epoch 1: val_accuracy improved from -inf to 0.18490, saving model to /content/Models
INFO:tensorflow:Assets written to: /content/Models/assets
552/552 [==============================] - 61s 92ms/step - loss: 1.6038 - accuracy: 0.5409 - val_loss: 3.5193 - val_accuracy: 0.1849
